# Read Files

press the >> symbol above to begin or restart. finally press restart and run all cells.

Alternatively. click on the first cell, or "Read Files". and keep pressing "> Run"

In [1]:
from IPython.core.display import display, HTML
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show inputs</button>"""))

In [2]:
import os
os.chdir("ExcelFiles")
print("local Excel Files: \n ")
for subdir, dirs, files in os.walk('./'):
    for file in files:
        if file[-4:] == "xlsx":
            print (file)

local Excel Files: 
 
July- Gallon Per Cow.xlsx
testdairydata.xlsx


In [3]:
#example implementation https://mybinder.org/v2/gh/SteveGrey759/MilkOScan/HEAD?filepath=fileopen%26graph.ipynb

In [4]:
#What global variables may be desired for future manipulation of this data?
#data grabbing:
#relevant sheet #, 
#data cleaning: 
#just general oversight over how cleaning pipeline process is going, with warnings signs for dramatic errors
#merging data:
#derived columns/frames:
#is days ago really a worthy column? 
#aggregates may take some interesting arguments
#graphs: 
#date ranges, minInclusion stats, graph types, etc


In [5]:
import numpy as np

In [6]:
fileName = ""
RawDairySheetName = "Raw Dairy (V#)"

In [7]:
import pandas as pd
import msoffcrypto 
import io 


while fileName != "arbitraryunlikelytext":
    fileName = input("Please enter the Excel files full name: ")
    #fileName = "testdairydata.xlsx"
    fileName = fileName.strip()
    password = input("Please enter the Excel password for DairyData: ")
    #password = ""

    try:
        file = msoffcrypto.OfficeFile(open(fileName, "rb"))
        file.load_key(password=password) # Use password
        decrypted = io.BytesIO() 
        file.decrypt(decrypted)
        RawDairy = pd.read_excel(decrypted, sheet_name=RawDairySheetName)
        fileName = "arbitraryunlikelytext"
        break
    except Exception as e: 
        print(e)
        print("Please try again, or restart")

In [8]:
#make all columns lowercase, and strip unecessary whitespaces
RawDairy.columns= RawDairy.columns.str.lower()
RawDairy.columns= RawDairy.columns.str.strip()
RawDairy.columns

Index(['location', 'material', 'category', 'subcategory', 'username',
       'analysis date/time', 'analysis id', 'sample id', 'comments',
       'igg, as-is %', 'moisture, %', 'igg, dry base %'],
      dtype='object')

In [9]:
#example of how to drop unnecessary columns
#MilkoscanPage2 = MilkoscanPage2.drop(columns = ['comments'])
#MilkoscanPage2.columns

# Cleaning raw dairy data: 
### Possible errors: 
* incorrect category (defatted, non V sampleid), 
* slightly incorrect entries (sampleid, date), 
* outliers. 
* repeated entries (accidentally entering same sample id twice, or systematically entering wrong sample IDs for a whole batch (off by one error) 

Delete Null/outlier values:
* Some columns have missing entries, or text such as "#N/A" or "Outlier"
* after converting those entries into a standard null value, we can display the number of nulls per column

In [10]:
RawDairy = RawDairy.replace('#N/A',np.nan)
RawDairy = RawDairy.replace('Outlier',np.nan)
print("Number of null values by column")
RawDairy.isnull().sum()

Number of null values by column


location                0
material                0
category                0
subcategory             0
username                0
analysis date/time      0
analysis id             0
sample id               0
comments              578
igg, as-is %            5
moisture, %             2
igg, dry base %         5
dtype: int64

### Delete all null entries from a specific column

In [11]:
def dropNullsFromColumn(df, column):
    df = df[df[column].notna()]
    return df

In [12]:
#dropNullsFromColumn(df = MilkoscanPage2, column = "moisture, %").isnull().sum() #it seems to work. 
for column in RawDairy.columns:
    if column not in ["comments","exampletext"]:
        RawDairy = dropNullsFromColumn(df = RawDairy, column = column)
RawDairy.isnull().sum() 

location                0
material                0
category                0
subcategory             0
username                0
analysis date/time      0
analysis id             0
sample id               0
comments              573
igg, as-is %            0
moisture, %             0
igg, dry base %         0
dtype: int64

### convert columns with small errors to type. delete outliers

In [13]:
RawDairy.dtypes

location                      object
material                      object
category                      object
subcategory                   object
username                      object
analysis date/time    datetime64[ns]
analysis id                   object
sample id                     object
comments                     float64
igg, as-is %                  object
moisture, %                  float64
igg, dry base %              float64
dtype: object

In [14]:
def coercedateTime(df, columnName):
    preNull = df[columnName].isnull().values.sum()
    dateTime = pd.to_datetime(df[columnName], errors = "coerce")
    #test.describe()
    print("coercing analysis date/time to datetime format has caused", dateTime.isnull().values.sum()-preNull ,"Number of null values")
    return dateTime
#test.isnull().values.sum()
RawDairy['analysis date/time'] = coercedateTime(df = RawDairy, columnName = 'analysis date/time')





coercing analysis date/time to datetime format has caused 0 Number of null values


In [15]:
def coerceNumericOrString(df, columnsToNumeric):
    #takes a list of column names to convert to numeric, converts all other object type columns to lowercase string
    for column in df.columns:
        if column in columnsToNumeric:
            print(column, "converted to float")
            df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')
        elif df[column].dtypes != "O":
            pass
            #already coerced into timestamp or other object
        else:
            df[column] = df[column].astype(str)
            df[column] = df[column].str.strip()
            df[column] = df[column].str.lower()
    return df
RawDairy = coerceNumericOrString(df = RawDairy, columnsToNumeric = ['igg, as-is %', "moisture, %", "igg, dry base %"])

igg, as-is % converted to float
moisture, % converted to float
igg, dry base % converted to float


Nulls created by coercing to type:

In [16]:
RawDairy.isnull().sum() 

location                0
material                0
category                0
subcategory             0
username                0
analysis date/time      0
analysis id             0
sample id               0
comments              573
igg, as-is %            2
moisture, %             0
igg, dry base %         0
dtype: int64

Again, drop all unacceptable nulls

In [17]:
for column in RawDairy.columns:
    if column not in ["comments","exampletext"]:
        RawDairy = dropNullsFromColumn(df = RawDairy, column = column)
RawDairy.isnull().sum() 

location                0
material                0
category                0
subcategory             0
username                0
analysis date/time      0
analysis id             0
sample id               0
comments              571
igg, as-is %            0
moisture, %             0
igg, dry base %         0
dtype: int64

### add future columns:

In [18]:
RawDairy["#tank"] = np.nan 

In [19]:
RawDairy["days ago"] = np.nan

### Delete incorrect category

#### removes all entries that have a sample id not starting with a V. Also shifts entries longer than 5 to a new column

 RawDairy['sample id'].max  #save shifting to new column for later date, discuss first

def moveExcess (x):
    x = str(x)
    if len(x) < 5:
        return x[3:]
    return np.nan
result = [moveExcess(x) for x in RawDairy['sample id']]
result

In [20]:
def filterNonRaw(x):
    #removes all entries that have a sample id not starting with a V.
    if x[0].lower() != "v":
        return np.nan
    if len(x) < 5:
        return np.nan
    return x[:5]
result = [filterNonRaw(x) for x in RawDairy['sample id']]
#RawDairy['sample id'] = result

In [21]:
#a sample of wrong ID samples
RawDairy[RawDairy['sample id'] != result].head(5)

,location,material,category,subcategory,username,analysis date/time,analysis id,sample id,comments,"igg, as-is %","moisture, %","igg, dry base %",#tank,days ago
60,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-05 11:53:00,az32538083,14353-4,NaN,2.48,76.699997,10.64,NaN,NaN
61,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-05 11:58:00,az44638083,14353-4,NaN,2.41,78.040001,10.97,NaN,NaN
62,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-05 12:00:00,az59638083,14353-5,NaN,1.99,78.010002,9.05,NaN,NaN
63,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-05 12:02:00,az56738083,14353-5,NaN,1.83,77.430000,8.11,NaN,NaN
64,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-05 12:20:00,az36048083,14353-6,NaN,1.82,78.309998,8.39,NaN,NaN


In [22]:
RawDairy['sample id'] = result

In [23]:
print(RawDairy['sample id'].isnull().sum(), "samples have invalid sample IDs")

33 samples have invalid sample IDs


In [24]:
RawDairy = dropNullsFromColumn(df = RawDairy, column = 'sample id')
#deleted entries with invalid sample IDs

### delete duplicate entries

In [25]:
RawDairy = RawDairy.sort_values(by=['analysis date/time'])
capacity = 5
suspiciousMinutes = 10 #minutes
popList = [] #[[name, date], [name, date]] #pop out an entry after it has exceeded a timelimit
duplicates = [] #[[412, 413],[442,443] ....]

for row, index in zip(RawDairy.itertuples(), range(len(RawDairy))) :
    date = row[6]
    name = row[8]
    iggAsIs = row[10]
    #print(row) #Pandas(Index=291, _1=Timestamp('2021-04-01 11:28:00'), _2='AZ43101083', _3=2.240000009536743, _4=80.08999633789062, _5=11.25, _6='DELBERT DENURE', _7=100.0, Region='WI')
    for recent in popList:
        if name == recent[0]: 
            if abs((recent[1] - date).total_seconds()) < 60*suspiciousMinutes: 
                duplicates.append(index)
                break
    else:
        popList.append([name, date]) 
    if len(popList) > capacity:
        popList = popList[1:]


print(len(duplicates))
print(duplicates[:20])
RawDairy.head(10)

120
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


,location,material,category,subcategory,username,analysis date/time,analysis id,sample id,comments,"igg, as-is %","moisture, %","igg, dry base %",#tank,days ago
0,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:28:00,az43101083,v4590,NaN,2.24,80.089996,11.25,NaN,NaN
1,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:30:00,az58101083,v4590,NaN,2.19,79.260002,10.56,NaN,NaN
2,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:32:00,az34201083,v4753,NaN,2.29,82.029999,12.74,NaN,NaN
3,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:33:00,az31301083,v4753,NaN,2.26,78.900002,10.71,NaN,NaN
4,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:35:00,az45301083,v4478,NaN,1.06,74.099998,4.09,NaN,NaN
5,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:38:00,az41401083,v4478,NaN,1.06,76.519997,4.51,NaN,NaN
6,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:40:00,az56401083,v4850,NaN,1.62,81.120003,8.58,NaN,NaN
7,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:41:00,az42501083,v4850,NaN,1.48,80.519997,7.60,NaN,NaN
8,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:43:00,az48501083,v4869,NaN,3.98,77.769997,17.90,NaN,NaN
9,"phoenix, az",colostrum,whole,any,adp@pantheryx,2021-04-01 11:45:00,az43601083,v4869,NaN,4.08,78.790001,19.24,NaN,NaN


Which columns are most subject to change? cow herd size? gallon per cow? 
we may keep these in a separate excel file

import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime

Read raw data from two files

RawDairy = pd.read_excel('raw-dairy-data.xlsx', sheet_name=0)